# Detección de fuego y caidas de personas con YOLOV8

## 1. Introducción y Objetivo del proyecto

El objetivo del proyecto es desarrollar un programa que detecte fuego y caidas de personas en tiempo real , cuando ocurra alguna de estas dos cosas nos debe avisar mandando un mensaje al teléfono móvil. Para la detección del fuego a tiempo real vamos a utilizar OpenCV con YOLO v8. Para que nos avise avise el telefono vamos emplear la API de telegram para recibir un mensaje a nuestro Telegram. Al detectar fuego, también se activara un alarma de emergencia de forma local. Para detectar la caida de personas vamos a utilizar el modelo de YOLOv8s sin entrenar y operaremos sobre el mismo en la clase personas.

Para la detección de fuego, podremos usar dos modelos preentrenados de YOLO v8:
- Modelo de detección de fuego.
- Modelo de detección de fuego y humo.

El programa esta pensado para la monitorización de casas donde viven personas dependientes, como ancianos o minusvalidos. 

## 2. Estructura del proyecto

El proyecto se divide en 3 partes:
- La primera parte es la instalación de librerias, dependencias y creación de funciones compartidas para la detección de fuego y caida.
- La segunda parte es la detección del fuego, donde primero tendremos un programa que visualiza en tiempo real si hay fuego o no y después se elaborara un programa donde se visualice en tiempo y real y avise por mensaje de texto y alarma local si hay fuego.
- La tercera parte es la detección de caida de personas, donde se desarrollara un programa para detectar la caida de personas en tiempo real y se añadirá la funcionalidad de que nos avise a través de un mensaje al teléfono.

## 3. Instalación de dependencias

### 3.1. Instalación de las librerias 

In [ ]:
pip install -r requirements.txt

### 3.2. Importación de las librerias

In [ ]:
from ultralytics import YOLO #importamos YOLO de la libreria de ultralytics para la detección de fuego
import cv2 #Importamos OpenCV para poder capturar las imagenes, visualizarlas y emplear YOLO en la detección de fuego
import threading #Importamos esta libreria para paralelizar procesos o funciones del programa en diferentes nucleos
import playsound #Importamos esta libreria para la función de alarma
import telebot #Importamos la libreria telebot para poder utilizar la API de Telegram y recibir el mensaje de alerta de fuego
import math #Importamos la liberia de matemáticas de python para calculos simples
import cvzone #Importamos la libreria CVZONE para poder trabajar de forma más sencilla con OpenCV y sobreescribir objetos visibles

### 3.3. Obtener Token de Telegram

Para obtener el ID de chat y el token de tu bot en Telegram, sigue estos pasos:

1. Crear un nuevo bot en BotFather:

-Abre Telegram y busca "BotFather".

-Inicia una conversación con BotFather.

-Usa el comando /newbot para crear un nuevo bot.

-Sigue las instrucciones de BotFather para asignar un nombre y un nombre de usuario a tu bot.

-BotFather te proporcionará un token único para tu bot. Guárdalo, ya que lo necesitarás para autorizar a tu script a enviar mensajes.

2. Obtener el ID de chat:

-Inicia una conversación con tu nuevo bot en Telegram.

-Envía cualquier mensaje al bot.

-Abre tu navegador web y accede a la siguiente URL, reemplazando <TOKEN> con el token de tu bot: 

-https://api.telegram.org/bot<TOKEN>/getUpdates

    -Ejemplo: https://api.telegram.org/bot1234567890:ABCDEFGHIJKLM/getUpdates
    
-Busca el campo "chat" en la respuesta JSON. El valor asociado con la clave "id" es el ID de chat.

3. Actualizar el script con la información obtenida:

-Sustituye 'tu_token_telegram' con el token que obtuviste de BotFather.

-Sustituye 'tu_id_chat_telegram' con el ID de chat que obtuviste al enviar un mensaje al bot.

Con estos pasos, tu script debería estar configurado para enviar mensajes a través de Telebot a tu cuenta de Telegram. Ten en cuenta que la URL para obtener actualizaciones (getUpdates) es solo para propósitos de obtener el ID de chat, y normalmente no se utiliza en producción. En un entorno de producción, deberías manejar las actualizaciones de manera diferente.

### 3.4. Función enviar mensaje Telegram

In [ ]:
# Token de tu bot en Telegram
telegram_token = 'tu_token_telegram' #Escribe aqui tu token

# Inicializar el bot de Telegram
bot = telebot.TeleBot(telegram_token)

# Función para enviar mensaje de fuego a Telegram utilizando Telebot
def enviar_mensaje_fuego_telegram():
    chat_id = 'tu_id_chat_telegram' #Escribe aqui tu chat_id
    message = "¡Fuego detectado! Por favor, revisa la transmisión de la cámara."

    # Enviar mensaje al chat especificado
    bot.send_message(chat_id, message)
    print("Mensaje de aviso de fuego enviado")
    
# Función para enviar mensaje de caida a Telegram utilizando Telebot
def enviar_mensaje_caida_persona():
    chat_id = 'tu_id_chat_telegram' #Escribe aqui tu chat_id
    message = "¡Se ha caido una persona! Por favor, revisa la transmisión de la cámara."

    # Enviar mensaje al chat especificado
    bot.send_message(chat_id, message)
    print("Mensaje de aviso de caida enviado")

### 3.5. Función alarma por fuego

In [ ]:
# Función definida para reproducir la alarma después de la detección de fuego mediante threading
def alarma_incendio():
    # Para reproducir la alarma, se proporciona también un archivo de audio MP3 con el código.
    playsound.playsound('Alarmas/alarm.mp3', True)

## 4. Detección de fuego

### 4.1. Programa con el modelo solo fuego

#### 4.1.1. Programa para solo visualizar si hay fuego (No ejecutar a la vez con los otros programas)

In [ ]:
if __name__ == '__main__':
    
    #Realizar videocaptura: Iniciar la cámara, "0" para la cámara incorporada y "1" para la cámara USB conectada.
    #cap = cv2.VideoCapture(0) #visualizar por la webcam en tiempo real 
    cap = cv2.VideoCapture("Videos/fire.mp4") #visualizar video grabado 1
    #cap = cv2.VideoCapture("Videos/fire2.mp4") #visualizar video grabado 2
    #cap = cv2.VideoCapture("Videos/fire3.mp4") #visualizar video grabado 3
    
    #Leer nuestro modelo
    model = YOLO("Modelos/fire_model.pt") #seleccionamos el modelo de YOLO y lo instanciamos

    while cap.isOpened():
        status, frame = cap.read() #iniciamos la captura de imagenes y la guardamos en la variable frame

        if not status: #si no hay captura de imagenes, sale del bucle
            break

        results = model.predict(frame) #pasamos los frames por modelo de YOLO y lo guardamos en la variable result
                                              
        frame = results[0].plot() #sobreescribimos la variable flame con los resultados ploteados (lo que el modelo detecta) si no tenemos stream activado

        cv2.imshow("Deteccion de fuego", frame) #mostramos el frame con los resultados ploteados
                  
        if cv2.waitKey(1) & 0xFF == ord("q"): #Para cerrar la visualización pulsar la letra "Q"
            break

    cap.release()
    cv2.destroyAllWindows()

#### 4.1.2. Programa detección de fuego con funciones (Visualización, mensajes y alarma)

In [ ]:
if __name__ == '__main__':
    
    #Realizar videocaptura: Iniciar la cámara, "0" para la cámara incorporada y "1" para la cámara USB conectada.
    #cap = cv2.VideoCapture(0) #visualizar por la webcam en tiempo real 
    cap = cv2.VideoCapture("Videos/fire.mp4") #visualizar video grabado 1
    #cap = cv2.VideoCapture("Videos/fire2.mp4") #visualizar video grabado 2
    #cap = cv2.VideoCapture("Videos/fire3.mp4") #visualizar video grabado 3
    
    #Leer nuestro modelo
    model = YOLO("Modelos/fire_model.pt") #seleccionamos el modelo de YOLO y lo instanciamos

    # Se listan las clases que tiene nuestro modelo (en el orden del data.yaml durante el entrenamiento)
    classnames = ['fire'] #Solo hay una clase, la clase 'fire' que es el numero [0] (id)
    
    # Contador global para la clase "fire"
    fire_count = 0
    
    while cap.isOpened():
        status, frame = cap.read() #iniciamos la captura de imagenes y la guardamos en la variable frame

        if not status: 
            break
            
        result = model.predict(frame, stream=True) # stream = True -> se utiliza para decir que vamos a iterar sobre los frames
        
        # Obtener la información de las bounding boxes (bboxes), la confianza y los nombres de clases detectados para operar con ellos. 
        for info in result:  #Este bucle for itera sobre los resultados de la inferencia realizada con el modelo YOLOv8. Cada elemento de result contiene información sobre las detecciones realizadas en el frame actual.
            boxes = info.boxes #Aquí se obtienen todas las bounding boxes detectadas en el frame actual y se almacenan en la variable boxes. 
                
            for box in boxes: #Este segundo bucle for itera sobre cada una de las bounding boxes detectadas.
                confidence = box.conf[0] #Se obtiene el valor de confianza de la detección actual y se almacena en la variable confidence.
                confidence = math.ceil(confidence * 100) #Se multiplica la confianza por 100 para obtener el porcentaje, y se redondea hacia arriba utilizando la función math.ceil().
                Class = int(box.cls[0]) #Se obtiene el índice de la clase detectada y se convierte a entero. Este valor se almacena en la variable Class.
                   
                if confidence > 10: #Se verifica si la confianza de la detección es mayor al 10%. Si es así, se procede a dibujar la bounding box y mostrar la información en el frame.
                    x1, y1, x2, y2 = box.xyxy[0] #Se obtienen las coordenadas de la bounding box (x1, y1, x2, y2) y se asignan a variables individuales.
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) #Se convierten las coordenadas a enteros.
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 5) #Se dibuja un rectángulo alrededor del objeto detectado en el frame, utilizando las coordenadas de la bounding box y el color rojo (0, 0, 255) con un grosor de 5 píxeles.
                    cvzone.putTextRect(frame, f'{classnames[Class]} {confidence}%', [x1 + 8, y1 + 100], scale=1.5, thickness=2) #Se agrega el nombre de la clase detectada y el porcentaje de confianza como texto en el frame, utilizando la biblioteca cvzone.
                    fire_count += 1 #aumentamos el contador
        
                    # Iniciar la alarma en un hilo
                    threading.Thread(target=alarma_incendio).start()

                    #Mandar mensaje por telegram
                    if fire_count == 1: #manda el mensaje si detecta que el contador vale 1
                        threading.Thread(target=enviar_mensaje_fuego_telegram).start()
                            
                    elif fire_count == 440: #El numero se elige dependiendo del tiempo que quieras entre mensajes, en este caso el número 440 son 60s aprox si el fuego es constante.
                        threading.Thread(target=enviar_mensaje_fuego_telegram).start()
                        fire_count = 2 #resetea el contador a 2        
            
        cv2.imshow("Deteccion de fuego", frame) #mostramos el frame con los resultados ploteados
                  
        if cv2.waitKey(1) & 0xFF == ord("q"): #Para cerrar la visualización pulsar la letra "Q"
            break

    cap.release()
    cv2.destroyAllWindows()

### 4.2. Programa con el modelo fuego y humo

#### 4.2.1. Programa para solo visualizar si hay fuego (No ejecutar a la vez con los otros programas)

In [ ]:
if __name__ == '__main__':
    
    #Realizar videocaptura: Iniciar la cámara, "0" para la cámara incorporada y "1" para la cámara USB conectada.
    #cap = cv2.VideoCapture(0) #visualizar por la webcam en tiempo real 
    cap = cv2.VideoCapture("Videos/fire.mp4") #visualizar video grabado 1
    #cap = cv2.VideoCapture("Videos/fire2.mp4") #visualizar video grabado 2
    #cap = cv2.VideoCapture("Videos/fire3.mp4") #visualizar video grabado 3
    
    #Leer nuestro modelo
    model = YOLO("Modelos/fire_model.pt") #seleccionamos el modelo de YOLO y lo instanciamos

    while cap.isOpened():
        status, frame = cap.read() #iniciamos la captura de imagenes y la guardamos en la variable frame

        if not status: #si no hay captura de imagenes, sale del bucle
            break

        results = model.predict(frame) #pasamos los frames por modelo de YOLO y lo guardamos en la variable result
                                              
        frame = results[0].plot() #sobreescribimos la variable flame con los resultados ploteados (lo que el modelo detecta) si no tenemos stream activado

        cv2.imshow("Deteccion de fuego", frame) #mostramos el frame con los resultados ploteados
                  
        if cv2.waitKey(1) & 0xFF == ord("q"): #Para cerrar la visualización pulsar la letra "Q"
            break

    cap.release()
    cv2.destroyAllWindows()

#### 4.2.2. Programa detección de fuego con funciones (Visualización, mensajes y alarma)

In [ ]:
if __name__ == '__main__':
    
    #Realizar videocaptura: Iniciar la cámara, "0" para la cámara incorporada y "1" para la cámara USB conectada.
    #cap = cv2.VideoCapture(0) #visualizar por la webcam en tiempo real 
    cap = cv2.VideoCapture("Videos/fire.mp4") #visualizar video grabado 1
    #cap = cv2.VideoCapture("Videos/fire2.mp4") #visualizar video grabado 2
    #cap = cv2.VideoCapture("Videos/fire3.mp4") #visualizar video grabado 3
    
    
    #Leer nuestro modelo
    model = YOLO("Modelos/fire_model_and_smoke.pt") #seleccionamos el modelo de YOLO y lo instanciamos

    # Se listan las clases que tiene nuestro modelo (en el orden del data.yaml durante el entrenamiento)
    classnames = ['fire', 'default', 'smoke'] # Ahora tenemos tres clases

    # Contador global para la clase "fire"
    fire_count = 0
    
    while cap.isOpened():
        status, frame = cap.read() #iniciamos la captura de imagenes y la guardamos en la variable frame

        if not status: 
            break
            
        result = model.predict(frame) # stream = True -> se utiliza para decir que vamos a iterar sobre los frames
        
        # Obtener la información de las bounding boxes (bboxes), la confianza y los nombres de clases detectados para operar con ellos. 
        for info in result:  
            boxes = info.boxes  # Ahora Class será 0 si se detecta 'fire', 1 si se detecta 'default', y 2 si se detecta 'smoke'
                
            for box in boxes: 
                confidence = box.conf[0] 
                confidence = math.ceil(confidence * 100) 
                Class = int(box.cls[0])
                   
                if confidence > 10: 
                    x1, y1, x2, y2 = box.xyxy[0] 
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) 
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 5) 
                        
                    if Class == 0: # Si se detecta 'fire'
                        fire_count += 1 #aumentamos el contador
                        cvzone.putTextRect(frame, f'{classnames[Class]} {confidence}%', [x1 + 8, y1 + 100], scale=1.5, thickness=2)
                        
                        # Iniciar la alarma en un hilo
                        threading.Thread(target=alarma_incendio).start()

                        #Mandar mensaje por telegram
                        if fire_count == 1: #manda el mensaje si detecta que el contador vale 1
                            threading.Thread(target=enviar_mensaje_fuego_telegram).start()
                            
                        elif fire_count == 440: #El numero se elige dependiendo del tiempo que quieras entre mensajes, en este caso el número 440 son 60s aprox si el fuego es constante.
                            threading.Thread(target=enviar_mensaje_fuego_telegram).start()
                            fire_count = 2 #resetea el contador a 2
                    
                    elif Class == 2: # Si se detecta 'smoke'
                        cvzone.putTextRect(frame, f'{classnames[Class]} {confidence}%', [x1 + 8, y1 + 100], scale=1.5, thickness=2)        
            
        cv2.imshow("Deteccion de fuego", frame) #mostramos el frame con los resultados ploteados
                  
        if cv2.waitKey(1) & 0xFF == ord("q"): #Para cerrar la visualización pulsar la letra "Q"
            break

    cap.release()
    cv2.destroyAllWindows()

## 5. Detección de caídas de personas

In [ ]:
if __name__ == '__main__':
    
    #Realizar videocaptura: Iniciar la cámara, "0" para la cámara incorporada y "1" para la cámara USB conectada.
    #cap = cv2.VideoCapture(0) #visualizar por la webcam en tiempo real 
    cap = cv2.VideoCapture("Videos/fall.mp4") #visualizar video grabado de caidas
    
    #Leer nuestro modelo
    model = YOLO("Modelos/yolov8s.pt") #seleccionamos el modelo de YOLO y lo instanciamos

    #Se listan las clases que tiene nuestro modelo (en el orden del archivo.txt con todas las clases del modelo)
    classnames = []
    with open('classes_model_fall_person.txt', 'r') as f:
        classnames = f.read().splitlines()
    
    # Contador global para las caidas
    fall_count = 0
    
    while cap.isOpened():
        status, frame = cap.read() #iniciamos la captura de imagenes y la guardamos en la variable frame

        if not status: #si no hay captura de imagenes, sale del bucle
            break

        results = model.predict(frame) #pasamos los frames por modelo de YOLO y lo guardamos en la variable result
        
        for info in results:
            parameters = info.boxes #guardamos en parameters la información de las bounding boxes
            for box in parameters:
                #ponemos que las nuevas coordenadas de la nueva bounding box sean la de la vieja
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                #guardamos la confianza del objeto detectado
                confidence = box.conf[0]
                #detectamos la clase del objeto detectado
                class_detect = box.cls[0]
                class_detect = int(class_detect)
                #Se utiliza el número de clase para obtener el nombre de la clase a partir de la lista classnames
                class_detect = classnames[class_detect]
                #Se calcula el porcentaje de confianza de la detección y se redondea hacia arriba.
                conf = math.ceil(confidence * 100)
        
        
                #Implementamos la detección de caida usando las coordenadas x1,y1,x2
                height = y2 - y1 #Se calcula la altura del cuadro delimitador.
                width = x2 - x1 #Se calcula el ancho del cuadro delimitador.
                threshold  = height - width #Se calcula la diferencia entre la altura y el ancho del cuadro delimitador.

                #Si la confianza es mayor al 80% y la clase detectada es 'persona' se plotea la detección
                if conf > 80 and class_detect == 'person': 
                    cvzone.cornerRect(frame, [x1, y1, width, height], l=30, rt=6) #Se dibuja un cuadro delimitador con esquinas redondeadas alrededor del objeto detectado.
                    cvzone.putTextRect(frame, f'{class_detect}', [x1 + 8, y1 - 12], thickness=2, scale=2) #Se muestra el nombre de la clase ("person") en el cuadro delimitador.

                #Si la diferencia entre la altura y el ancho del cuadro delimitador es negativa, se indica una posible caída.
                if threshold < 0:
                    cvzone.putTextRect(frame, 'Caida detectada', [height, width], thickness=2, scale=2)
                    fall_count += 1
                    
                    #Mandar mensaje por telegram
                    if fall_count == 10: #manda el mensaje si detecta que el contador vale 10
                        threading.Thread(target=enviar_mensaje_caida_persona).start()
                            
                    elif fall_count == 100: #El numero se elige dependiendo del tiempo que quieras entre mensajes, en este caso el número 100 es cuando la persona esta unos 2 segundos en el suelo.
                        threading.Thread(target=enviar_mensaje_caida_persona).start()
                        fall_count = 11 #resetea el contador a 11
                        
                #Si no pasa    
                else:pass

        cv2.imshow("Deteccion de caidas de personas", frame) #mostramos el frame con los resultados ploteados
                  
        if cv2.waitKey(1) & 0xFF == ord("q"): #Para cerrar la visualización pulsar la letra "Q"
            break

    cap.release()
    cv2.destroyAllWindows()

## 6. Bibliografia

Link github del modelo de solo fuego: https://github.com/salim4n/Fire_Detection?tab=readme-ov-file

Link del github del modelo de fuego y humo: https://github.com/Abonia1/YOLOv8-Fire-and-Smoke-Detection/tree/main

Link del github del modelo de detección de caida de personas: https://github.com/Tech-Watt/Fall-Detection/tree/main